In [1]:
import pandas as pd
import json
import requests
import time
import threading
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from sqlalchemy import create_engine

In [2]:
# 取得 Token , 設置請求參數 (大陸)
url = "https://login.xiaoshouyi.com/auc/oauth2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {
    "grant_type": "password",
    "client_id": "473f31e3a6cb9cafaec3ddf36d20092b",
    "client_secret": "ddfed65f5aeebacd0f54eaa464fcb043",
    "redirect_uri": "https://api-scrm.xiaoshouyi.com/",
    "username": "11021041@twkd.com",
    "password": "Marcellin01oiH88cCI"
}

# 發送 post 請求
response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    print(response.json())
else:
    print("請求失敗, 狀態碼:", response.status_code)

content = response.json()
ac_token = content["access_token"]


print(ac_token)

{'access_token': '9ff1845453462f45c7eb15587937ae5d594f3948d95e9af46daeebb7cc511c70.MjM2OTI2Njk5ODU2MzExMw==0', 'token_type': 'Bearer', 'expires_in': 86399, 'scope': 'all prm crm', 'tenant_id': '2369266998563113', 'login_type': '', 'encryption_key': '', 'client_id': '473f31e3a6cb9cafaec3ddf36d20092b', 'passport_id': '3052172005298442', 'mobile_params': '', 'instance_uri': 'api-scrm.xiaoshouyi.com'}
9ff1845453462f45c7eb15587937ae5d594f3948d95e9af46daeebb7cc511c70.MjM2OTI2Njk5ODU2MzExMw==0


In [3]:
'''
select from user 人員名單
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
} 
data = {
    "xoql": "select id, name, dimDepart from user",
    #"xoql": "insert into contact(contactName, phone, email) values('貴夫人', '0912345678', 'kkdd@gmail.com')"
    "batchCount": 2000
}
response2 = requests.post(url_2, headers=headers, data = data)
ownerId = response2.json()
print(ownerId)
user_df = pd.json_normalize(ownerId["data"]["records"])    

# user_df[user_df['id']=='']
# user_df[user_df['name']=='']

{'code': '200', 'msg': '', 'errorInfo': None, 'debugInfo': None, 'data': {'totalSize': 630, 'count': 630, 'records': [{'id': '2369301140133416', 'name': 'CRM管理员', 'dimDepart': '2586653553808230'}, {'id': '2508839248500196', 'name': '谢敏', 'dimDepart': '2369301140133408'}, {'id': '2508849255192015', 'name': '唐雪梅', 'dimDepart': '2746547845976859'}, {'id': '2531364166931931', 'name': '俞丽', 'dimDepart': '2369301140133408'}, {'id': '2531364279604701', 'name': '宋冀泽', 'dimDepart': '2369301140133408'}, {'id': '2586658357285641', 'name': '徐玮珍', 'dimDepart': '2586637755765606'}, {'id': '2586659192656747', 'name': '杨卫卫', 'dimDepart': '2586653553808212'}, {'id': '2586660964537152', 'name': '金海洋', 'dimDepart': '2586653553808230'}, {'id': '2586661145334617', 'name': '教育训练', 'dimDepart': '2746546870686577'}, {'id': '2586661881402202', 'name': '蔡家维', 'dimDepart': '2369301140133408'}, {'id': '2586663184339728', 'name': '刘俐君', 'dimDepart': '2586649840522022'}, {'id': '2586670417466176', 'name': '马梦文', 'd

In [4]:
'''
select related_contact 客戶關係聯絡人
'''

url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
contact_related = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
        SELECT 
        contactCode__c 联系人代号, 
        id 客户联系人关系, 
        customItem25__c 关系状态,
        dimDepart, 
        customItem38__c 联系人代号_舊,
        customItem119__c 有效_总,
        account__c.customItem335__c 经销商签约系列
        FROM accContactRelation__c 
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    contact_related = pd.concat([contact_related, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

# 有效(总) 為 有效
contact_related['有效_总']= contact_related['有效_总'].astype(str)
contact_related = contact_related[~contact_related['有效_总'].str.contains("无效")]

contact_related['经销商签约系列'] = contact_related['经销商签约系列'].astype(str)

contact_related = contact_related[~contact_related['经销商签约系列'].str.contains("全系列", na=False)]

In [ ]:
# from datetime import datetime

# # Unix 时间戳（以毫秒为单位）
# timestamp_ms = 1706630400000

# # 将毫秒转换为秒
# timestamp_s = timestamp_ms / 1000

# # 使用 datetime.fromtimestamp() 方法将 Unix 时间戳转换为日期
# date = datetime.fromtimestamp(timestamp_s)

# print(date)

In [ ]:
# from datetime import datetime

# # 输入日期
# date_string = "2023-03-02"
# date_format = "%Y-%m-%d"

# # 将日期字符串解析为 datetime 对象
# date = datetime.strptime(date_string, date_format)

# # 将 datetime 对象转换为 Unix 时间戳（以毫秒为单位）
# timestamp_ms = int(date.timestamp() * 1000)

# print(timestamp_ms)

In [7]:
'''
select from trackingrecord 追蹤紀錄
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}

date_to_convert = datetime(2024, 1, 1) 
# date_to_convert = datetime(2023, 3, 2) # 大陸銷售易上線日期
timestamp = int(date_to_convert.timestamp() * 1000)

queryLocator = ''
tracking = pd.DataFrame()
while True:
    data = {
        "xoql": f'''
        SELECT
        name 业务工作记录代号,
        customItem55__c.contactCode__c 联系人代号,
        customItem55__c.customItem38__c 联系人代号_旧,
        customItem45__c 最近工作内容,
        customItem132__c 无效电访类型,
        customItem106__c 电访接通无效,
        customItem115__c 是否邀约K大,
        customItem40__c 最近联系时间,
        customItem4__c 触客类型,
        customItem55__c 客户联系人关系, 
        dimDepart.departName 资料区域名称,
        customItem59__c 联系人,
        customItem27__c customItem70__c,
        accountCode__c 公司代号
        FROM customEntity15__c
        WHERE customItem40__c >= {timestamp}
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
        
    }
    
    response2 = requests.post(url_2, headers=headers, data = data)
    exist = response2.json()
    data = pd.DataFrame(exist["data"]["records"])
    tracking = pd.concat([tracking, data], ignore_index=True, sort=False)
    
    if not exist['queryLocator']:
        break
    queryLocator = exist['queryLocator']
    
tracking['最近联系时间'] = tracking['最近联系时间'].astype(float)
tracking['最近联系时间'] = tracking['最近联系时间'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
tracking['最近联系时间'] = tracking['最近联系时间'].dt.tz_convert('Asia/Taipei')
tracking['最近联系时间'] = tracking['最近联系时间'].dt.strftime('%Y-%m-%d')
tracking = tracking.sort_values(by='最近联系时间', ascending=True) 

current_datetime = datetime.now()
target_date = current_datetime - timedelta(days=90) 

tracking['无效电访类型']= tracking['无效电访类型'].astype(str)
tracking['电访接通无效']= tracking['电访接通无效'].astype(str)
tracking['最近联系时间'] = pd.to_datetime(tracking['最近联系时间'])

# 三个月内联系过
tracking_month = tracking[tracking['最近联系时间'] >= target_date] 

# 联络人退休/转行/公司 & 电话暂停/空号/号码错误
tracking_invalid = tracking[(tracking['无效电访类型'].str.contains("联络人|电话"))]

# 号码无效
tracking_invalid_2 = tracking[(tracking['电访接通无效'].str.contains("号码无效|智齒"))]

# 提取 tracking_invalid 中的聯絡人代號
existing_contact_codes = tracking_invalid['联系人代号']

# 检查 tracking_invalid_2 中的聯絡人代號是否存在于 tracking_invalid 中
mask = ~tracking_invalid_2['联系人代号'].isin(existing_contact_codes)

# 过滤出符合条件的行
tracking_invalid_2 = tracking_invalid_2[mask]

# 智齒攔截名單
tracking_zhichi = tracking[(tracking['电访接通无效'].str.contains("智齒"))]

K_invite_rejected = pd.concat([tracking_month, tracking_invalid, tracking_invalid_2])

In [8]:
經營開發 = pd.read_excel("/Users/marcel/KD科定/CN_K大/經營開發_all_20250224.xlsx", dtype='object')
經營開發 = pd.merge(經營開發, tracking, on = '联系人代号', how = 'left')
經營開發 = 經營開發.sort_values(by='最近联系时间', na_position='first')
經營開發 = 經營開發[~經營開發['联系人代号'].isin(K_invite_rejected['联系人代号'].drop_duplicates())]
經營開發 = 經營開發.drop_duplicates(subset=['联系人代号'], keep='last')

In [ ]:
# url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/xobjects/customEntity14__c/busiType"
# headers = {
#     "Authorization": f"Bearer {ac_token}"
# }
# response2 = requests.get(url_2, headers = headers)
# Task_desc = response2.json()

# Task_desc

In [9]:
'''
from 銷售易 其他電拜訪(原:電/拜訪)

# 已给出的交办，但没有及时处理
1、交办管理中【业务类型】：业务每月交办（业管指定）、行销指定产品顾问邀约K大交办、发放回访交办
2、交办管理中【审批状态】：审批中

# 短信群发回复有意向客户名单
1、交办管理中【业务类型】：行销指定产品顾问邀约K大交办
2. 交办管理中【工作主旨】：短信回复需要样册

'''

url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
Tasks_df = pd.DataFrame()

while True:
    data = {
        "xoql": f'''
        SELECT 
        name, 
        customItem42__c.contactCode__c 联系人代号, 
        createdAt 创建日期,
        customItem3__c 工作主旨, 
        customItem8__c 执行状态,
        approvalStatus 审批状态
        FROM 
            customEntity14__c
        WHERE 
            (entityType = '2825954839057885' OR entityType = '2954421329187625' OR entityType = '3220592941503278')
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Tasks_df = pd.concat([Tasks_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

Tasks_df['创建日期'] = Tasks_df['创建日期'].astype(float)
Tasks_df['创建日期'] = Tasks_df['创建日期'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
Tasks_df['创建日期'] = Tasks_df['创建日期'].dt.tz_convert('Asia/Taipei')
Tasks_df['创建日期'] = Tasks_df['创建日期'].dt.strftime('%Y-%m-%d')
Tasks_df['创建日期'] = pd.to_datetime(Tasks_df['创建日期'])
# Tasks_df['执行状态']= Tasks_df['执行状态'].astype(str)
# Tasks_df1 = Tasks_df.loc[Tasks_df['执行状态'].str.contains("任务完成|接受任务")]  

Tasks_df['审批状态']= Tasks_df['审批状态'].astype(str)
Tasks_df1 = Tasks_df.loc[Tasks_df['审批状态'].str.contains("待提交") | Tasks_df['审批状态'].str.contains("审批中")]  

Tasks_df = Tasks_df.loc[Tasks_df['工作主旨'].str.contains("短信")]  



In [10]:
K_invite = pd.merge(經營開發, contact_related, on = '联系人代号', how = 'inner')
K_invite = K_invite.drop_duplicates(subset=['联系人代号'], keep='last')

K_invite = K_invite[~K_invite['联系人代号'].isin(Tasks_df['联系人代号'].drop_duplicates())]
K_invite = K_invite[~K_invite['联系人代号'].isin(Tasks_df1['联系人代号'].drop_duplicates())]
# K_invite = K_invite[~K_invite['联系人代号'].isin(Tasks_daily['联系人代号'].drop_duplicates())]
K_invite = K_invite.rename(columns={'联系人_x':'联系人'})

In [ ]:

# url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
# headers = {
#     "Authorization": f"Bearer {ac_token}",
#     "Content-Type":"application/x-www-form-urlencoded"
# }
# queryLocator = ''
# Tasks_daily = pd.DataFrame()

# while True:
#     data = {
#         "xoql": f'''
#         SELECT 
#         name, 
#         customItem42__c.contactCode__c 联系人代号, 
#         createdAt 创建日期,
#         customItem3__c 工作主旨, 
#         customItem8__c 执行状态,
#         approvalStatus 审批状态
#         FROM 
#             customEntity14__c
#         WHERE 
#             entityType = '3074631761565155'
#         ''',
#         "batchCount": 2000,
#         "queryLocator": queryLocator
#     }
#     response = requests.post(url_2, headers=headers, data=data)
#     crm = response.json()
#     data = pd.DataFrame(crm["data"]["records"])
#     Tasks_daily = pd.concat([Tasks_daily, data], ignore_index=True, sort=False)
    
#     if not crm['queryLocator']:
#         break
#     queryLocator = crm['queryLocator']

# Tasks_daily['创建日期'] = Tasks_daily['创建日期'].astype(float)
# Tasks_daily['创建日期'] = Tasks_daily['创建日期'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
# Tasks_daily['创建日期'] = Tasks_daily['创建日期'].dt.tz_convert('Asia/Taipei')
# Tasks_daily['创建日期'] = Tasks_daily['创建日期'].dt.strftime('%Y-%m-%d')
# Tasks_daily['创建日期'] = pd.to_datetime(Tasks_daily['创建日期']).dt.date

# current_date = current_datetime.date()

# Tasks_daily = Tasks_daily[Tasks_daily['创建日期'] >= current_date] 

# Tasks_daily

In [11]:
# 分公司信息 customEntity32__c
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
sales_dimDepart = pd.DataFrame()

while True:
    data = {
        "xoql": '''
        SELECT 
        name 区域, 
        customItem47__c.name 电访人员
        FROM 
            customEntity32__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    sales_dimDepart = pd.concat([sales_dimDepart, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

sales_dimDepart = sales_dimDepart.drop_duplicates(subset=['区域', '电访人员'])
sales_dimDepart = sales_dimDepart[~(sales_dimDepart['区域'].str.contains('加盟商') | sales_dimDepart['区域'].str.contains('天猫') | sales_dimDepart['区域'].str.contains('经销商'))]
sales_dimDepart = sales_dimDepart[sales_dimDepart['电访人员'] != ""]

In [12]:
# 合併人員資訊表格
K_invite = K_invite.rename(columns={'资料区域名称_x':'资料区域名称','联系人_x':'联系人'})
K_invite = K_invite.drop(['customItem70__c'],axis=1)
K_invite = pd.merge(K_invite , sales_dimDepart ,left_on ='资料区域名称', right_on='区域', how='left')
K_invite = K_invite[~K_invite['资料区域名称'].str.contains("经销商|加盟商")]

# K_invite['電訪人員'].fillna('姚思瑋', inplace=True)

# 手機電話去重、加回
K_invite1 = K_invite[K_invite['联系人手机号'].apply(lambda x: isinstance(x, str) and len(x) == 11 and x != '00000000000')] 
K_invite1 = K_invite1.drop_duplicates(subset=['联系人手机号'], keep='last')

K_invite2 = K_invite[~(K_invite['联系人手机号'].isin(K_invite1['联系人手机号'].drop_duplicates(keep=False)))]
contact_related['关系状态']= contact_related['关系状态'].astype(str)
contact_resigned = contact_related[contact_related['关系状态'].str.contains('离职')]
K_invite2 = K_invite2[~(K_invite2['联系人代号'].isin(contact_resigned['联系人代号'].drop_duplicates(keep=False)))]

K_invite_all = pd.concat([K_invite1,K_invite2])

In [13]:
'''
from K大預約表 排除2024/5/26以來K過的(大陸美耐板上線)
'''

url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
K_visit_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
        SELECT 
        name,
        customItem29__c 联系人代号, 
        customItem6__c 客戶關係連絡人編號,
        customItem2__c 预约日期, 
        customItem40__c, 
        customItem9__c 联系人手机号
        FROM customEntity26__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    K_visit_df = pd.concat([K_visit_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

# # 從大陸銷售易上線那天起所有K大場次    
# K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Taipei')
# start_date = pd.Timestamp('2023-03-02', tz='UTC') # 大陸销售易上线日期
# K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'])
# K_visit_df = K_visit_df[K_visit_df['预约日期'] >= start_date]

# # 近三個月的K大場次
# K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Taipei')
# end_date = pd.Timestamp.now(tz='Asia/Taipei')
# start_date = end_date - pd.DateOffset(months=3)
# K_visit_df = K_visit_df[(K_visit_df['预约日期'] >= start_date) & (K_visit_df['预约日期'] < end_date)]

# 排除2024/5/26以來K過的
K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Taipei')
start_date = pd.Timestamp('2024-05-26', tz='UTC') # 大陸美耐板上線
K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'])
K_visit_df = K_visit_df[K_visit_df['预约日期'] >= start_date]

# 有舉行K大
K_visit_df['customItem40__c'] = K_visit_df['customItem40__c'].astype(str)
K_visit_df_held = K_visit_df[K_visit_df['customItem40__c'].str.contains("是")]

# 未來場次
today_date = datetime.now().strftime('%Y-%m-%d')
K_visit_df_future = K_visit_df[K_visit_df['预约日期'] >= today_date]

K_visit_df = pd.concat([K_visit_df_held, K_visit_df_future])


'''
from K大預約参會人
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
K_visit_df_attendant = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
        select 
        name,
        customItem1__c K大预约表编号,
        customItem6__c 联系人代号, 
        customItem5__c 联系人手机号
        from customEntity27__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    K_visit_df_attendant = pd.concat([K_visit_df_attendant, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
K_visit_df = pd.concat([K_visit_df, K_visit_df_attendant])

# # 排除K過3次及以上的人
# K_visit_df1 = K_visit_df.groupby('联系人手机号').size().reset_index(name='次數')
# K_visit_df1 = K_visit_df1[(K_visit_df1['联系人手机号'] != '') & (K_visit_df1['次數'] >= 3)]

# K_visit_df2 = K_visit_df.groupby('客戶關係連絡人編號').size().reset_index(name='次數')
# K_visit_df2 = K_visit_df2[(K_visit_df2['客戶關係連絡人編號'] != '') & (K_visit_df2['次數'] >= 3)]

# K_invite_all = K_invite_all[~K_invite_all['联系人手机号'].isin(K_visit_df1['联系人手机号'].drop_duplicates())]
# K_invite_all = K_invite_all[~K_invite_all['客户联系人关系'].isin(K_visit_df2['客戶關係連絡人編號'].drop_duplicates())]

# 去除已K大+已邀約
K_invite_all = K_invite_all[~K_invite_all['联系人代号'].isin(K_visit_df['联系人代号'].drop_duplicates())]
K_invite_all = K_invite_all[~K_invite_all['联系人手机号'].isin(K_visit_df['联系人手机号'].drop_duplicates())]

K_invite_all = K_invite_all.rename(columns={'accountId':'customItem70__c'})
K_invite_all = K_invite_all.rename(columns={'联系人_x':'联系人'})


/var/folders/sb/5kcp5c2n3fg2g6ywrjr9_f580000gn/T/ipykernel_95674/233602618.py:50: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Taipei')


In [14]:
'''
from K大預約表 (排除K過3次及以上的人)
'''

url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
K_visit_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''
        SELECT 
        name,
        customItem29__c 联系人代号, 
        customItem6__c 客戶關係連絡人編號,
        customItem2__c 预约日期, 
        customItem40__c, 
        customItem9__c 联系人手机号
        FROM customEntity26__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    K_visit_df = pd.concat([K_visit_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

# 從大陸銷售易上線那天起所有K大場次    
K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Taipei')
start_date = pd.Timestamp('2023-03-02', tz='UTC') # 大陸销售易上线日期
K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'])
K_visit_df = K_visit_df[K_visit_df['预约日期'] >= start_date]

# 有舉行K大
K_visit_df['customItem40__c'] = K_visit_df['customItem40__c'].astype(str)
K_visit_df_held = K_visit_df[K_visit_df['customItem40__c'].str.contains("是")]

# 未來場次
today_date = datetime.now().strftime('%Y-%m-%d')
K_visit_df_future = K_visit_df[K_visit_df['预约日期'] >= today_date]

K_visit_df = pd.concat([K_visit_df_held, K_visit_df_future])

# 排除K過3次及以上的人
K_visit_df1 = K_visit_df.groupby('联系人手机号').size().reset_index(name='次數')
K_visit_df1 = K_visit_df1[(K_visit_df1['联系人手机号'] != '') & (K_visit_df1['次數'] >= 3)]

K_visit_df2 = K_visit_df.groupby('客戶關係連絡人編號').size().reset_index(name='次數')
K_visit_df2 = K_visit_df2[(K_visit_df2['客戶關係連絡人編號'] != '') & (K_visit_df2['次數'] >= 3)]

K_invite_all = K_invite_all[~K_invite_all['联系人手机号'].isin(K_visit_df1['联系人手机号'].drop_duplicates())]
K_invite_all = K_invite_all[~K_invite_all['客户联系人关系'].isin(K_visit_df2['客戶關係連絡人編號'].drop_duplicates())]

/var/folders/sb/5kcp5c2n3fg2g6ywrjr9_f580000gn/T/ipykernel_95674/2429273170.py:38: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  K_visit_df['预约日期'] = pd.to_datetime(K_visit_df['预约日期'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Taipei')


In [16]:
'''
select from Tasks 
台湾 : entityType = '3028348436713387' or '3077984163073940 (每日K大邀約電訪交辦 or 釣魚簡訊交辦)
大陸 : entityType = '3074631761565155' (每日K大邀約電訪交辦)
'''

url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
Tasks_df = pd.DataFrame()

date_to_convert = datetime(2024, 1, 31) 
timestamp = int(date_to_convert.timestamp() * 1000)

while True:
    data = {
        "xoql": f'''
        SELECT 
        customItem42__c.contactCode__c 联系人代号, 
        customItem58__c 电访接通_无效,
        customItem8__c 执行状态
        FROM customEntity14__c
        WHERE 
            entityType = '3074631761565155' 
        AND 
            createdAt >= {timestamp}
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Tasks_df = pd.concat([Tasks_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

Tasks_df['执行状态']= Tasks_df['执行状态'].astype(str)    
Tasks_df['电访接通_无效']= Tasks_df['电访接通_无效'].astype(str)  
Tasks_df = Tasks_df.loc[Tasks_df['执行状态'].str.contains("任务完成|进行中")]
Tasks_df1 = Tasks_df.loc[Tasks_df['电访接通_无效'].str.contains("已做开场介绍|客户拒绝邀约|未做开场介绍|客户挂断")]

K_invite_all = K_invite_all[~K_invite_all['联系人代号'].isin(Tasks_df1['联系人代号'].drop_duplicates())]

K_invite_all = K_invite_all.drop_duplicates(subset=['联系人代号'], keep='last')
K_invite_all = K_invite_all.drop_duplicates(subset=['联系人手机号'], keep='last')
K_invite_all = K_invite_all.sort_values(by='最近联系时间', na_position='first') 
K_invite_all['entityType'] = '3074631761565155'
today_date = datetime.now().strftime('%Y-%m-%d')
K_invite_all['customItem1__c'] = today_date # 台灣:期望完成日, 大陸:期望時間
K_invite_all['customItem3__c'] = 'K大邀约' + K_invite_all['联系人'] + "第一次电访"



In [17]:
# 經營客戶
K_invite_promote = K_invite_all.loc[K_invite_all['目标客户类型'].str.contains("经营", na=False)]
K_invite_promoting = K_invite_all.loc[K_invite_all['目标客户类型'].str.contains("开发中", na=False)]

# 非經營客戶
K_invite_non_promote = K_invite_all[~K_invite_all['联系人代号'].isin(K_invite_promote['联系人代号'].drop_duplicates())]
K_invite_non_promote = K_invite_non_promote[~K_invite_non_promote['联系人代号'].isin(K_invite_promoting['联系人代号'].drop_duplicates())]

# K_invite_promoting = K_invite_promoting.loc[K_invite_promoting['区域'].str.contains("上海", na=False)]
K_invite_non_promote = K_invite_non_promote.loc[K_invite_non_promote['区域'].str.contains("上海", na=False)]


K_invite_經營_count = K_invite_promote['联系人代号'].count()
K_invite_開發中_count = K_invite_promoting['联系人代号'].count()
K_invite_開發_count = K_invite_non_promote['联系人代号'].count()

# K_invite_經營 = (K_invite_經營_count / (K_invite_經營_count + K_invite_開發_count) )
# K_invite_開發中 = (K_invite_開發中_count / (K_invite_經營_count + K_invite_開發中_count+ K_invite_開發_count) )
# K_invite_開發 = 1 - K_invite_經營 - K_invite_開發中

K_invite_經營 = 0.7
K_invite_開發中 = 0.2
K_invite_開發 = 0.1

print(K_invite_經營)
print(K_invite_開發中)
print(K_invite_開發)

0.7
0.2
0.1


In [18]:
'''
select from Sales list K大顧問暨電訪人員名單
customItem7__c 國家/區域
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}
queryLocator = ''
Sales_list = pd.DataFrame()

while True:
    data = {
        "xoql": '''
        SELECT 
        customItem2__c 电访人员, 
        customItem4__c 预估通数, 
        customItem3__c 电访人员类型
        FROM customEntity47__c
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Sales_list = pd.concat([Sales_list, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

Sales_list['电访人员类型']= Sales_list['电访人员类型'].astype(str)
Sales_list = Sales_list[Sales_list['预估通数']!='']
Sales_list['预估通数']= Sales_list['预估通数'].astype(int)
Sales_list = pd.merge(Sales_list, user_df, left_on = '电访人员', right_on = 'id', how = 'left')
# Sales_list = Sales_list.loc[~Sales_list['name'].isin(['黃祿庭', '陳鵬宇', '周淑真', '葉子瑄', '黃詩婷', '林育君'])] # 改為負責智齒攔截

Sales_list = Sales_list[Sales_list['预估通数'] > 0]
Sales_list1 = Sales_list.loc[~Sales_list['电访人员类型'].str.contains("产品顾问")]
Sales_list2 = Sales_list.loc[Sales_list['电访人员类型'].str.contains("产品顾问")]
Sales_list1['经营'] = K_invite_經營 * Sales_list1['预估通数']
Sales_list1['開發中'] = K_invite_開發中 * Sales_list1['预估通数']
Sales_list1['開發'] = K_invite_開發 * Sales_list1['预估通数']
Sales_list2['经营'] = K_invite_經營 * Sales_list2['预估通数']
Sales_list2['開發中'] = K_invite_開發中 * Sales_list2['预估通数']
Sales_list2['開發'] = K_invite_開發 * Sales_list2['预估通数']
Sales_list1['经营'] = Sales_list1['经营'].astype(int)
Sales_list1['開發中'] = Sales_list1['開發中'].astype(int)
Sales_list1['開發'] = Sales_list1['開發'].astype(int)
Sales_list2['经营'] = Sales_list2['经营'].astype(int)
Sales_list2['開發中'] = Sales_list2['開發中'].astype(int)
Sales_list2['開發'] = Sales_list2['開發'].astype(int)


/var/folders/sb/5kcp5c2n3fg2g6ywrjr9_f580000gn/T/ipykernel_95674/794047257.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sales_list1['经营'] = K_invite_經營 * Sales_list1['预估通数']
/var/folders/sb/5kcp5c2n3fg2g6ywrjr9_f580000gn/T/ipykernel_95674/794047257.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Sales_list1['開發中'] = K_invite_開發中 * Sales_list1['预估通数']
/var/folders/sb/5kcp5c2n3fg2g6ywrjr9_f580000gn/T/ipykernel_95674/794047257.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [19]:
###########接區，如當區名單不夠，隨機分配(要依照經營開發比例給名單)，同未接區邏輯
K_invite接區1 = []
for _, row in Sales_list2.iterrows():
    people = row['name']
    count = row['经营']
    
    K_invite_promote.reset_index(drop=True, inplace=True)

    count = int(count)
    
    matching_rows = K_invite_promote[K_invite_promote['电访人员'] == people].head(count)
    
    K_invite接區1.append(matching_rows)

K_invite接區1 = pd.concat(K_invite接區1)
K_invite接區1.reset_index(drop=True, inplace=True)

K_invite_promote = K_invite_promote[~K_invite_promote['联系人代号'].isin(K_invite接區1['联系人代号'].drop_duplicates())]

K_invite接區2 = []
for _, row in Sales_list2.iterrows():
    people = row['name']
    count = row['開發中']

    K_invite_promoting.reset_index(drop=True, inplace=True)
    
    count = int(count)
    
    matching_rows = K_invite_promoting[K_invite_promoting['电访人员'] == people].head(count)
    
    K_invite接區2.append(matching_rows)

K_invite接區2 = pd.concat(K_invite接區2)
K_invite接區2.reset_index(drop=True, inplace=True)

K_invite_promoting = K_invite_promoting[~K_invite_promoting['联系人代号'].isin(K_invite接區2['联系人代号'].drop_duplicates())]

K_invite接區3 = []
for _, row in Sales_list2.iterrows():
    people = row['name']
    count = row['開發']
    
    K_invite_non_promote.reset_index(drop=True, inplace=True)
    
    count = int(count)
    
    matching_rows = K_invite_non_promote[K_invite_non_promote['电访人员'] == people].head(count)
    
    K_invite接區3.append(matching_rows)

K_invite接區3 = pd.concat(K_invite接區3)
K_invite接區3.reset_index(drop=True, inplace=True)

K_invite_non_promote = K_invite_non_promote[~K_invite_non_promote['联系人代号'].isin(K_invite接區3['联系人代号'].drop_duplicates())]

K_invite_district = pd.concat([K_invite接區1,K_invite接區2, K_invite接區3])
# K_invite_district = pd.concat([K_invite接區1, K_invite接區3])
K_invite_district = K_invite_district.sample(frac=1).reset_index(drop=True)
K_invite_district = K_invite_district.drop_duplicates(subset=['联系人代号'], keep='last')

K_invite_district = pd.merge(K_invite_district, user_df, left_on = '电访人员', right_on = 'name', how = 'left')
K_invite_district = pd.merge(K_invite_district, user_df, left_on = '区域', right_on = 'name', how = 'left')

K_invite_district = K_invite_district.drop(['电访人员'],axis=1)
K_invite_district = K_invite_district.drop(['dimDepart'],axis=1)
K_invite_district = K_invite_district.drop(['dimDepart_y'],axis=1)
K_invite_district = K_invite_district.rename(columns={'id_x':'电访人员', 'dimDepart_x' : 'dimDepart', 'id_y':'ownerId'})

K_invite_district['电访人员'].value_counts()

电访人员
2644467339219785    250
2644467588813655    150
2644467588813632    150
Name: count, dtype: int64

In [20]:
#########未接區
invited_people1 = []

for index, row in Sales_list1.iterrows():
    person = row['电访人员']  # Assuming 'person' is the column name
    count = row['经营']  # Assuming 'count' is the column name

    # Extend the invited_people list with 'person' repeated 'count' times
    if not pd.isna(count):
        invited_people1.extend([person] * int(count))
invited_people1 = pd.DataFrame({'客服主任': invited_people1})

invited_people2 = []

for index, row in Sales_list1.iterrows():
    person = row['电访人员']  # Assuming 'person' is the column name
    count = row['開發中']  # Assuming 'count' is the column name

    # Extend the invited_people list with 'person' repeated 'count' times
    if not pd.isna(count):
        invited_people2.extend([person] * int(count))
invited_people2 = pd.DataFrame({'客服主任': invited_people2})

invited_people3 = []

for index, row in Sales_list1.iterrows():
    person = row['电访人员']  # Assuming 'person' is the column name
    count = row['開發']  # Assuming 'count' is the column name

    # Extend the invited_people list with 'person' repeated 'count' times
    if not pd.isna(count):
        invited_people3.extend([person] * int(count))
invited_people3 = pd.DataFrame({'客服主任': invited_people3})


invited_people1.reset_index(drop=True, inplace=True)
invited_people2.reset_index(drop=True, inplace=True)
invited_people3.reset_index(drop=True, inplace=True)
K_invite_promote.reset_index(drop=True, inplace=True)
K_invite_promoting.reset_index(drop=True, inplace=True)
K_invite_non_promote.reset_index(drop=True, inplace=True)

# filtered_K_invite_promote = K_invite_promote[~K_invite_promote['区域'].isin(['泉州', '厦门', '西安', '南宁', '昆明、贵阳', '重庆', '成都', '无锡', '苏州'])]
# filtered_K_invite_promote.reset_index(drop=True, inplace=True)  
K_invite未接1 = pd.concat([K_invite_promote,invited_people1],axis=1)
K_invite未接1.reset_index(drop=True, inplace=True)  
K_invite未接1 = K_invite未接1.drop(['电访人员'],axis=1)
K_invite未接1 = K_invite未接1.rename(columns={'客服主任':'电访人员'})
K_invite未接1 = K_invite未接1[K_invite未接1['电访人员'].notnull()]
K_invite未接1 = K_invite未接1[K_invite未接1['联系人代号'].notnull()]
K_invite_promote = K_invite_promote[~K_invite_promote['联系人代号'].isin(K_invite未接1['联系人代号'].drop_duplicates())]

# filtered_K_invite_promoting = K_invite_promoting[~K_invite_promoting['区域'].isin(['泉州', '厦门', '西安', '南宁', '昆明、贵阳', '重庆', '成都', '无锡', '苏州'])]
# filtered_K_invite_promoting.reset_index(drop=True, inplace=True)  
K_invite未接2 = pd.concat([K_invite_promoting,invited_people2],axis=1)
K_invite未接2.reset_index(drop=True, inplace=True)  
K_invite未接2 = K_invite未接2.drop(['电访人员'],axis=1)
K_invite未接2 = K_invite未接2.rename(columns={'客服主任':'电访人员'})
K_invite未接2 = K_invite未接2[K_invite未接2['电访人员'].notnull()]
K_invite未接2 = K_invite未接2[K_invite未接2['联系人代号'].notnull()]
K_invite_promoting = K_invite_promoting[~K_invite_promoting['联系人代号'].isin(K_invite未接2['联系人代号'].drop_duplicates())]

# filtered_K_invite_non_promote = K_invite_non_promote[~K_invite_non_promote['区域'].isin(['泉州', '厦门', '西安', '南宁', '昆明、贵阳', '重庆', '成都', '无锡', '苏州'])]
# filtered_K_invite_non_promote.reset_index(drop=True, inplace=True)  
K_invite未接3 = pd.concat([K_invite_non_promote,invited_people3],axis=1)
K_invite未接3.reset_index(drop=True, inplace=True)  
K_invite未接3 = K_invite未接3.drop(['电访人员'],axis=1)
K_invite未接3 = K_invite未接3.rename(columns={'客服主任':'电访人员'})
K_invite未接3 = K_invite未接3[K_invite未接3['电访人员'].notnull()]
K_invite未接3 = K_invite未接3[K_invite未接3['联系人代号'].notnull()]
K_invite_non_promote = K_invite_non_promote[~K_invite_non_promote['联系人代号'].isin(K_invite未接3['联系人代号'].drop_duplicates())]

K_invite_non_district = pd.concat([K_invite未接1,K_invite未接2,K_invite未接3])
# K_invite_non_district = pd.concat([K_invite未接1, K_invite未接3])
K_invite_non_district = K_invite_non_district.sample(frac=1).reset_index(drop=True)
K_invite_non_district = K_invite_non_district.drop_duplicates(subset=['联系人代号'], keep='last')

K_invite_non_district = pd.merge(K_invite_non_district, user_df, left_on = '区域', right_on = 'name', how = 'left')
K_invite_non_district = K_invite_non_district.drop(['dimDepart_y'],axis=1)
K_invite_non_district = K_invite_non_district.rename(columns={'id':'ownerId', 'dimDepart_x' : 'dimDepart'})

K_invite_non_district['电访人员'].value_counts()

电访人员
3321172257968944    300
3708943322109405    300
3715951347749679    300
2709514453554019    200
Name: count, dtype: int64

In [21]:
K_invite_list = pd.concat([K_invite_non_district, K_invite_district])

K_invite_list['customItem11__c'] = K_invite_list['customItem70__c']

K_invite_list = K_invite_list.drop_duplicates(subset=['联系人代号'], keep='last')

K_invite_list.rename(columns={'客户联系人关系':'customItem42__c',
                              '电访人员':'customItem10__c'}, inplace=True)

K_invite_task = K_invite_list[['customItem42__c',
                            'entityType',
                            'customItem1__c',
                            'customItem3__c',
                            'customItem10__c',
                            'dimDepart',
                            'customItem70__c',
                            'customItem11__c',
                            'ownerId']]

K_invite_task = K_invite_task.astype(str)


K_invite_task

,customItem42__c,entityType,customItem1__c,customItem3__c,customItem10__c,dimDepart,customItem70__c,customItem11__c,ownerId
0,2836097501419053,3074631761565155,2025-03-20,K大邀约朱妍第一次电访,3321172257968944,2794755975468573,2756243355784718,2756243355784718,2897694514025223
1,2836097200133678,3074631761565155,2025-03-20,K大邀约孙仁恒(R)第一次电访,2709514453554019,2794756346320423,2756232035112469,2756232035112469,2826974397371875
2,2836134191453741,3074631761565155,2025-03-20,K大邀约潘伶俐第一次电访,3708943322109405,2794743622637072,2756223858709987,2756223858709987,2928339093246734
3,2836103486461478,3074631761565155,2025-03-20,K大邀约王乐皎第一次电访,3708943322109405,2794755975468573,2756225346639318,2756225346639318,2897694514025223
4,2836121351607773,3074631761565155,2025-03-20,K大邀约周代翔R第一次电访,3708943322109405,2794755576436161,2668687754515266,2668687754515266,2957364629590860
...,...,...,...,...,...,...,...,...,...
545,3434409692975895,3074631761565155,2025-03-20,K大邀约刘成杨第一次电访,2644467588813632,2794755360101904,3434405452731153,3434405452731153,3573225808221641
546,2836129936037355,3074631761565155,2025-03-20,K大邀约邹德成R第一次电访,2644467339219785,2794758458791434,2756243286365714,2756243286365714,2957372554761024
547,2836150756578776,3074631761565155,2025-03-20,K大邀约郭海第一次电访,2644467339219785,2794758458791434,2756224563516870,2756224563516870,2957372554761024
548,2836147728815630,3074631761565155,2025-03-20,K大邀约涂涂第一次电访,2644467339219785,2794748093863381,2756244523865538,2756244523865538,2957376513676107


In [ ]:
##########補名單

# K_invite_add_all = pd.read_excel('K_invite_add_all.xlsx')
K_invite_add_all = pd.concat([K_invite_promote, K_invite_promoting, K_invite_non_promote])
K_invite_add_all = K_invite_add_all[~K_invite_add_all['联系人代号'].isin(K_invite_list['联系人代号'].drop_duplicates())]
K_invite_add_all.to_excel('K_invite_add_all.xlsx', index=False)

In [ ]:
Sales = "张超"

K_invite_add_all = K_invite_add_all.sample(frac=1).reset_index(drop=True)

# # 根據區域提供名單
# K_invite_add_all = K_invite_add_all[~K_invite_add_all['区域'].isna()]
# K_invite_add_all = K_invite_add_all[K_invite_add_all['区域'].str.contains('杭州')]

# 補名單數量
K_invite_add = K_invite_add_all[:300]
K_invite_add_all = K_invite_add_all[~K_invite_add_all['联系人代号'].isin(K_invite_add['联系人代号'].drop_duplicates())]

K_invite_add.loc[K_invite_add['电访人员'] != '', '电访人员'] = Sales

K_invite_task = pd.merge(K_invite_add, user_df, left_on = '电访人员', right_on = 'name', how = 'left')
K_invite_task = pd.merge(K_invite_task, user_df, left_on = '区域', right_on = 'name', how = 'left')


K_invite_task = K_invite_task.drop(['电访人员'],axis=1)
K_invite_task = K_invite_task.drop(['dimDepart'],axis=1)
K_invite_task = K_invite_task.rename(columns={'id_x':'电访人员'})

K_invite_task.rename(columns={'电访人员': 'customItem10__c',
                              '客户联系人关系':'customItem42__c',
                              'dimDepart_x':'dimDepart',
                              'id_y':'ownerId'}, inplace=True)

K_invite_task['customItem11__c'] = K_invite_task['customItem70__c']

K_invite_task = K_invite_task[['customItem42__c',
                               'entityType',
                               'customItem1__c',
                               'customItem3__c',
                               'customItem10__c',
                               'dimDepart',
                               'customItem70__c',
                               'customItem11__c',
                               'ownerId']]

K_invite_task = K_invite_task.astype(str)

K_invite_task


In [22]:
'''
ask bulk id
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/job"

headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/json"
}

data = {
     "data": {
        "operation": "insert",####################################################################3
        "object": "customEntity14__c",
        "execOption": ["CHECK_RULE", "CHECK_DUPLICATE"]
    }
    
}

response = requests.post(url_2, headers = headers, json = data)
test = response.json()
bulk_id = test["result"]["id"]
print(bulk_id)


e9fa96a8caa871e0ab36de8779d0f0cf40cff787062642a085cf5f52131e5788


In [23]:
'''
insert to crm if dataframe over 5000
'''
batch_size = 5000

# Calculate the number of batches
num_batches = (len(K_invite_task) + batch_size - 1) // batch_size

# Initialize a list to store responses
all_responses = []

# Initialize an index to keep track of the current batch
current_batch_index = 0

# While there are more batches to process
while current_batch_index < num_batches:
    start_index = current_batch_index * batch_size
    end_index = (current_batch_index + 1) * batch_size
    current_batch_df = K_invite_task.iloc[start_index:end_index]
    json_data = current_batch_df.to_dict(orient='records')
    
    url_2 = "https://api-scrm.xiaoshouyi.com/rest/bulk/v2/batch"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type":"application/json"
    }
    data = {
        "data": {
            "jobId": f"{bulk_id}",
            "datas": json_data
        }
    }
    response = requests.post(url_2, headers=headers, json=data)
    all_responses.append(response)
    current_batch_index += 1
for response in all_responses:
    pass

In [26]:
'''
select from Tasks 
'''
url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"

headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
}

queryLocator = ''

Tasks_df = pd.DataFrame()

while True:
    data = {
        "xoql": '''
        SELECT 
        id,
        name, 
        customItem10__c, 
        customItem3__c, 
        approvalStatus
        FROM customEntity14__c 
        WHERE 
            entityType = '3074631761565155'
        AND
            createdBy = '3052177351988706'
        ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Tasks_df = pd.concat([Tasks_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
Tasks_df['approvalStatus']= Tasks_df['approvalStatus'].astype(str)
Tasks_df = Tasks_df.loc[Tasks_df['approvalStatus'].str.contains("待提交")] 

Tasks_df

,id,name,customItem10__c,customItem3__c,approvalStatus


In [25]:
'''
submit_task
'''

status_url = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/task/actions/preProcessor"
# task_url = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type": "application/json"
    }

def preProcessor(process_ID):
    status_body = {
        "data": {
            "action": "submit",
            "entityApiKey": "customEntity14__c",
            "dataId": process_ID }}

    response = requests.post(status_url, headers=headers, json=status_body)
    crm_json = response.json()['data']
    return crm_json

tasks_df_last_row = Tasks_df.iloc[-1]
approval_status = preProcessor(tasks_df_last_row['id'])

def submit_task(row):
    data_id = row['id']
    task_id = row['customItem10__c']
    url_2 = "https://api-scrm.xiaoshouyi.com/rest/data/v2.0/creekflow/task"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type": "application/json"
    }

    data = {
        "data": {
            "action": "submit",
            "entityApiKey": "customEntity14__c",
            "dataId": data_id,
            "procdefId": approval_status['procdefId'],
            "nextTaskDefKey": approval_status['nextTaskDefKey'],
            "nextAssignees": [task_id],
            "ccs": [task_id]  
        }
    }

    response = requests.post(url_2, headers=headers, json=data)
    result = response.json()
    print(f"Response for dataId {data_id}: {result}")

# Assuming data_ids is your DataFrame
data_ids_df = Tasks_df[['id', 'customItem10__c']]

# Create a thread for each row in the DataFrame
threads = []
for index, row in data_ids_df.iterrows():
    thread = threading.Thread(target=submit_task, args=(row,))
    threads.append(thread)
    thread.start()
    time.sleep(0.08)

# Wait for all threads to finish
for thread in threads:
    thread.join()
    


Response for dataId 3730080054235947: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730080054235948: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730080054235950: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730080054235951: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730080054235949: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730080054235952: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730080054235953: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730081343641054: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730081343641055: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
Response for dataId 3730081343641057: {'code': '200', 'msg': 'OK', 'data': None, 'errorInfo': None}
